<a href="https://colab.research.google.com/github/KoheiMinoda/Ocean_Engineering/blob/main/barge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
from scipy import pi
import math
from scipy.linalg import solve

In [2]:
# Constants
N_MAX = 30
ZI = 0 + 1j  # Complex number (0.0, 1.0)

# Arrays
ipvts = np.zeros(N_MAX + 1, dtype=int)
ipvta = np.zeros(N_MAX + 1, dtype=int)
k2 = np.zeros(N_MAX + 1, dtype=float)
n1 = np.zeros(N_MAX + 1, dtype=float)
n2 = np.zeros(N_MAX + 1, dtype=float)
wkn = np.zeros(N_MAX, dtype=float)
g2 = np.zeros(N_MAX + 1, dtype=float)
g3 = np.zeros(N_MAX + 1, dtype=float)
g4 = np.zeros(N_MAX + 1, dtype=float)
g7 = np.zeros(N_MAX + 1, dtype=float)
c = np.zeros((N_MAX + 1, N_MAX + 1), dtype=float)
fs = np.zeros((N_MAX + 1, N_MAX + 1), dtype=float)
fa = np.zeros((N_MAX + 1, N_MAX + 1), dtype=float)
err = np.zeros(5, dtype=float)
k3 = np.zeros(N_MAX + 1, dtype=float)
n3 = np.zeros(N_MAX + 1, dtype=float)
rmats = np.zeros((N_MAX + 1, N_MAX + 1), dtype=float)
rmata = np.zeros((N_MAX + 1, N_MAX + 1), dtype=float)
rg1 = np.zeros(N_MAX + 1, dtype=float)
rg2 = np.zeros(N_MAX + 1, dtype=float)
rg3 = np.zeros(N_MAX + 1, dtype=float)
rg4 = np.zeros(N_MAX + 1, dtype=float)
rg5 = np.zeros(N_MAX + 1, dtype=float)
rg6 = np.zeros(N_MAX + 1, dtype=float)
rg7 = np.zeros(N_MAX + 1, dtype=float)
ra2 = np.zeros(N_MAX + 1, dtype=float)
ra3 = np.zeros(N_MAX + 1, dtype=float)
ra4 = np.zeros(N_MAX + 1, dtype=float)
rba = np.zeros((N_MAX + 1, 2), dtype=float)
rb2 = np.zeros(N_MAX + 1, dtype=float)
rb3 = np.zeros(N_MAX + 1, dtype=float)
rb4 = np.zeros(N_MAX + 1, dtype=float)
k1 = np.zeros(N_MAX + 1, dtype=complex)
mats = np.zeros((N_MAX + 1, N_MAX + 1), dtype=complex)
mata = np.zeros((N_MAX + 1, N_MAX + 1), dtype=complex)
ass = np.zeros(N_MAX + 1, dtype=complex)
aaa = np.zeros(N_MAX + 1, dtype=complex)
bs = np.zeros(N_MAX + 1, dtype=complex)
ba = np.zeros(N_MAX + 1, dtype=complex)
a2 = np.zeros(N_MAX + 1, dtype=complex)
a3 = np.zeros(N_MAX + 1, dtype=complex)
a4 = np.zeros(N_MAX + 1, dtype=complex)
b2 = np.zeros(N_MAX + 1, dtype=complex)
b3 = np.zeros(N_MAX + 1, dtype=complex)
b4 = np.zeros(N_MAX + 1, dtype=complex)
g1 = np.zeros(N_MAX + 1, dtype=complex)
g5 = np.zeros(N_MAX + 1, dtype=complex)
g6 = np.zeros(N_MAX + 1, dtype=complex)
zbs = np.zeros((N_MAX + 1, 2), dtype=complex)
zba = np.zeros((N_MAX + 1, 3), dtype=complex)
za = np.zeros((3, 3), dtype=complex)
zf = np.zeros(3, dtype=complex)

# Constants
PI = np.pi

In [3]:
# Input fluid density, rho (Mg/m^3)
rho = float(input('Input fluid density, rho(Mg/m^3): '))

# Input gravity acceleration, g (m/s^2)
g = float(input('Input gravity acceleration, g(m/s^2): '))

# Input full width of the barge, B (m)
bb = float(input('Input full width of the barge, B(m): '))

# Input draft of the barge, d (m)
d = float(input('Input draft of the barge, d(m): '))

# Input water depth, h (m)
h = float(input('Input water depth, h(m): '))

# Input height of the center of gravity, KG (m)
kg = float(input('Input height of the center of gravity, KG(m): '))

# Input radius of gyration, rx (m)
rx = float(input('Input radius of gyration, rx(m): '))

# Input interval of frequencies, df (Hz)
df = float(input('Input interval of frequencies, df(Hz): '))

# Input number of frequencies, nf
nf = int(input('Input number of frequencies, nf: '))

Input fluid density, rho(Mg/m^3): 1.025
Input gravity acceleration, g(m/s^2): 9.80665
Input full width of the barge, B(m): 30.0
Input draft of the barge, d(m): 2.5
Input water depth, h(m): 8.0
Input height of the center of gravity, KG(m): 1.75
Input radius of gyration, rx(m): 9.0
Input interval of frequencies, df(Hz): 0.001
Input number of frequencies, nf: 1000


In [4]:
nn = N_MAX
b = bb / 2
zg = kg - d
m = rho * bb * d
gm = d/2 + bb**2/(12*d) - kg

subroutine

In [5]:
def CALB(k1, b, nn, N_MAX, c, g2, g4, aas, aaa, a2, a3, a4):

    for n in range(nn+1):
        bs[n] = 0.5 * np.exp(k1[0] * b) * c[n, 0]
        ba[n] = 0.5 * np.exp(k1[0] * b) * c[n, 0]
        b2[n] = 0.0
        b3[n] = -g2[n]
        b4[n] = -g4[n]

        for l in range(nn+1):
            bs[n] += aas[l] * c[n, l]
            ba[n] += aaa[l] * c[n, l]
            b2[n] += a2[l] * c[n, l]
            b3[n] += a3[l] * c[n, l]
            b4[n] += a4[l] * c[n, l]

    return bs, ba, b2, b3, b4

In [6]:
def CALRB(b, nn, N_MAX, c, g2, g4, a2, a3, a4, b2, b3, b4):

    for n in range(nn + 1):
        b2[n] = 0.0
        b3[n] = -g2[n]
        b4[n] = -g4[n]

        for l in range(nn + 1):
            b2[n] += a2[l] * c[n, l]
            b3[n] += a3[l] * c[n, l]
            b4[n] += a4[l] * c[n, l]

    return b2, b3, b4

In [7]:
def FORCE(rho, g, h, b, nn, k_1, n1, k2, n2, g1, g6, g7, aaa, ba, bs):

    F2 = 0.5 * np.exp(ZI * k_1 * b) * g1[0] + aaa[0] * g1[0]
    F3 = b * bs[0] / n2[0]
    F4 = 0.5 * h**2 * np.exp(ZI * k_1 * b) * g6[0] + h**2 * aaa[0] * g6[0] + ba[0] * b**2 / (3.0 * n2[0])

    for n in range(1, nn + 1):
        F2 += aaa[n] * g1[n]
        F3 += bs[n] * (-1)**n * np.tanh(k2[n] * b) / (k2[n] * n2[n])
        F4 += h**2 * aaa[n] * g6[n] + (-1)**n * ba[n] / n2[n] * g7[n]

    F2 = -2 * rho * g * n1[0] * h / np.cosh(k_1 * h) * F2
    F3 = 2 * rho * g * n1[0] / np.cosh(k_1 * h) * F3
    F4 = 2 * rho * g * n1[0] / np.cosh(k_1 * h) * F4

    return F2, F3, F4


In [8]:
def ADMAS(rho, h, d, b, nn, k2, n2, g1, g6, g7, a2, a4, b2, b3, b4):

    a22 = a2[0] * g1[0]
    a33 = h * b * b3[0] / n2[0]
    a44 = h**2 * a4[0] * g6[0] + b**2 * b4[0] / (3.0 * n2[0])
    a24 = a4[0] * g1[0]
    a42 = h**2 * a2[0] * g6[0] + b**2 * b2[0] / (3.0 * n2[0])

    for n in range(1, nn + 1):
        a22 += a2[n] * g1[n]
        a33 += h * (-1)**n * np.tanh(k2[n] * b) * b3[n] / (n2[n] * k2[n])
        a44 += h**2 * a4[n] * g6[n] + (-1)**n * b4[n] / n2[n] * g7[n]
        a24 += a4[n] * g1[n]
        a42 += h**2 * a2[n] * g6[n] + (-1)**n * b2[n] / n2[n] * g7[n]

    a22 = -rho * 2 * h**2 * a22
    a33 = rho * 2 * (a33 + ((h - d)**2 * b - b**3 / 3.0) / (2 * (h - d)))
    a44 = rho * 2 * h**2 * (a44 + b**2 * (b * (h - d)**2 - b**3 / 5.0) / (6 * h**2 * (h - d)))
    a24 = -rho * 2 * h**3 * a24
    a42 = rho * 2 * h * a42

    return a22, a33, a44, a24, a42


In [9]:
def RADMAS(rho, h, d, b, nn, k2, n2, g1, g6, g7, a2, a4, b2, b3, b4):

    a22 = a2[0] * g1[0]
    a33 = h * b * b3[0] / n2[0]
    a44 = h**2 * a4[0] * g6[0] + b**2 * b4[0] / (3.0 * n2[0])
    a24 = a4[0] * g1[0]
    a42 = h**2 * a2[0] * g6[0] + b**2 * b2[0] / (3.0 * n2[0])

    for n in range(1, nn + 1):
        a22 += a2[n] * g1[n]
        a33 += h * (-1)**n * np.tanh(k2[n] * b) * b3[n] / (n2[n] * k2[n])
        a44 += h**2 * a4[n] * g6[n] + (-1)**n * b4[n] / n2[n] * g7[n]
        a24 += a4[n] * g1[n]
        a42 += h**2 * a2[n] * g6[n] + (-1)**n * b2[n] / n2[n] * g7[n]

    a22 = -rho * 2 * h**2 * a22
    a33 = rho * 2 * (a33 + ((h - d)**2 * b - b**3 / 3.0) / (2 * (h - d)))
    a44 = rho * 2 * h**2 * (a44 + b**2 * (b * (h - d)**2 - b**3 / 5.0) / (6 * h**2 * (h - d)))
    a24 = -rho * 2 * h**3 * a24
    a42 = rho * 2 * h * a42

    return a22, a33, a44, a24, a42


In [10]:
def VECTR(h, b, nn, N_MAX, k2, C, g1, g2, g3, g4, g5, g6, a2, a3, a4):

    for m in range(nn + 1):
        zsum3 = 0.0
        zsum4 = g4[0] * h / b * C[0, m]

        for l in range(1, nn + 1):
            zsum3 += g2[l] * k2[l] * h * np.tanh(k2[l] * b) * C[l, m]
            zsum4 += g4[l] * k2[l] * h / np.tanh(k2[l] * b) * C[l, m]

        a2[m] = -g1[m]
        a3[m] = zsum3 + g3[m]
        a4[m] = zsum4 - g5[m] + g6[m]

In [11]:
def RVECTR(h, b, nn, N_MAX, k2, C, g1, g2, g3, g4, g5, g6, a2, a3, a4):

    for m in range(nn + 1):
        zsum3 = 0.0
        zsum4 = g4[0] * h / b * C[0, m]

        for l in range(1, nn + 1):
            zsum3 += g2[l] * k2[l] * h * np.tanh(k2[l] * b) * C[l, m]
            zsum4 += g4[l] * k2[l] * h / np.tanh(k2[l] * b) * C[l, m]

        a2[m] = -g1[m]
        a3[m] = zsum3 + g3[m]
        a4[m] = zsum4 - g5[m] + g6[m]


In [12]:
def MATRX(h, b, nn, N_MAX, k1, fs, fa, mats, mata, ass, aaa):

    def DELT(N, M):
        if N == M:
            return 1.0
        else:
            return 0.0

    for m in range(nn + 1):
        ass[m] = 0.5 * np.exp(k1[0] * b) * (DELT(m, 0) * k1[m] * h - fs[m, 0])
        aaa[m] = 0.5 * np.exp(k1[0] * b) * (DELT(m, 0) * k1[m] * h - fa[m, 0])

        for l in range(nn + 1):
            mats[m, l] = DELT(m, l) * k1[m] * h + fs[m, l]
            mata[m, l] = DELT(m, l) * k1[m] * h + fa[m, l]


In [13]:
def RMATRX(h, b, nn, N_MAX, k1, fs, fa, mats, mata):

    def DELT(N, M):
        if N == M:
            return 1.0
        else:
            return 0.0

    for m in range(nn + 1):
        for l in range(nn + 1):
            mats[m, l] = DELT(m, l) * k1[m] * h + fs[m, l]
            mata[m, l] = DELT(m, l) * k1[m] * h + fa[m, l]

In [14]:
def CALGM(h, d, b, zg, nn, k1, k2, n1, n2):

    g2[0] = 1.0 / (2 * h**2 * (h - d) * n2[0]) * ((h - d)**3 / 3 - b**2 * (h - d))
    g4[0] = b / (6 * h**3 * (h - d) * n2[0]) * ((h - d)**3 - b**2 * (h - d))
    g7[0] = 0.0

    for n in range(1, nn + 1):
        g2[n] = 1.0 / (2 * h**2 * (h - d) * n2[n]) * (
            1.0 / k2[n] * ((h - d)**2 - 2.0 / k2[n]**2) * np.sin(k2[n] * (h - d))
            + 2 * (h - d) / k2[n]**2 * np.cos(k2[n] * (h - d)) - b**2 / k2[n] * np.sin(k2[n] * (h - d))
        )
        g4[n] = b / (2 * h**3 * (h - d) * n2[n]) * (
            1.0 / k2[n] * ((h - d)**2 - 2.0 / k2[n]**2) * np.sin(k2[n] * (h - d))
            + 2 * (h - d) / k2[n]**2 * np.cos(k2[n] * (h - d)) - b**2 / (3 * k2[n]) * np.sin(k2[n] * (h - d))
        )
        g7[n] = b**2 * (1.0 / np.tanh(k2[n] * b) / (k2[n] * b) - 1.0 / (k2[n] * b)**2)

    for n in range(nn + 1):
        g1[n] = (np.sin(k1[n] * h) - np.sin(k1[n] * (h - d))) / (n1[n] * k1[n] * h)
        g3[n] = b / (h * (h - d)) / (n1[n] * k1[n]) * np.sin(k1[n] * (h - d))
        g5[n] = 1.0 / (2 * h**2 * (h - d) * n1[n]) * (
            1.0 / k1[n] * ((h - d)**2 - 2.0 / k1[n]**2) * np.sin(k1[n] * (h - d))
            + 2 * (h - d) / k1[n]**2 * np.cos(k1[n] * (h - d)) - b**2 / k1[n] * np.sin(k1[n] * (h - d))
        )
        g6[n] = 1.0 / (h**2 * n1[n]) * (
            1.0 / k1[n]**2 * np.cos(k1[n] * h) + d / k1[n] * np.sin(k1[n] * (h - d))
            - 1.0 / k1[n]**2 * np.cos(k1[n] * (h - d))
        ) - zg / h * g1[n]

    return g1, g2, g3, g4, g5, g6, g7


In [15]:
def RCALGM(h, d, b, zg, nn, k1, k2, n1, n2):

    g2[0] = 1.0 / (2 * h**2 * (h - d) * n2[0]) * ((h - d)**3 / 3 - b**2 * (h - d))
    g4[0] = b / (6 * h**3 * (h - d) * n2[0]) * ((h - d)**3 - b**2 * (h - d))
    g7[0] = 0.0

    for n in range(1, nn + 1):
        g2[n] = 1.0 / (2 * h**2 * (h - d) * n2[n]) * (
            1.0 / k2[n] * ((h - d)**2 - 2.0 / k2[n]**2) * np.sin(k2[n] * (h - d))
            + 2 * (h - d) / k2[n]**2 * np.cos(k2[n] * (h - d)) - b**2 / k2[n] * np.sin(k2[n] * (h - d))
        )
        g4[n] = b / (2 * h**3 * (h - d) * n2[n]) * (
            1.0 / k2[n] * ((h - d)**2 - 2.0 / k2[n]**2) * np.sin(k2[n] * (h - d))
            + 2 * (h - d) / k2[n]**2 * np.cos(k2[n] * (h - d)) - b**2 / (3 * k2[n]) * np.sin(k2[n] * (h - d))
        )
        g7[n] = b**2 * (1.0 / np.tanh(k2[n] * b) / (k2[n] * b) - 1.0 / (k2[n] * b)**2)

    for n in range(nn + 1):
        g1[n] = (np.sin(k1[n] * h) - np.sin(k1[n] * (h - d))) / (n1[n] * k1[n] * h)
        g3[n] = b / (h * (h - d)) / (n1[n] * k1[n]) * np.sin(k1[n] * (h - d))
        g5[n] = 1.0 / (2 * h**2 * (h - d) * n1[n]) * (
            1.0 / k1[n] * ((h - d)**2 - 2.0 / k1[n]**2) * np.sin(k1[n] * (h - d))
            + 2 * (h - d) / k1[n]**2 * np.cos(k1[n] * (h - d)) - b**2 / k1[n] * np.sin(k1[n] * (h - d))
        )
        g6[n] = 1.0 / (h**2 * n1[n]) * (
            1.0 / k1[n]**2 * np.cos(k1[n] * h) + d / k1[n] * np.sin(k1[n] * (h - d))
            - 1.0 / k1[n]**2 * np.cos(k1[n] * (h - d))
        ) - zg / h * g1[n]

    return g1, g2, g3, g4, g5, g6, g7


In [16]:
def CALFSA(h, b, nn, N_MAX, k2, c):

    fs = np.empty((N_MAX + 1, nn + 1), dtype=float)
    fa = np.empty((N_MAX + 1, nn + 1), dtype=float)

    for m in range(nn + 1):
        for n in range(nn + 1):
            zsum2 = 0.0
            zsum4 = 0.0
            for l in range(1, nn + 1):
                zsum2 += k2[l] * h * np.tanh(k2[l] * b) * c[l, m] * c[l, n]
                zsum4 += k2[l] * h / np.tanh(k2[l] * b) * c[l, m] * c[l, n]

            fs[m, n] = zsum2
            fa[m, n] = zsum4 + h / b * c[0, m] * c[0, n]

    return fs, fa


In [17]:
def COEF2(h, d, nn, N_MAX, k1, k2, n1, n2):

    k1 = np.zeros(nn+1, dtype=complex)
    C = np.empty((N_MAX + 1, nn + 1), dtype=complex)

    for m in range(nn + 1):
        for n in range(nn + 1):
            C[m, n] = 1.0 / (2 * h * n2[m] * n1[n]) * (
                1.0 / (k2[m] - k1[n]) * np.sin((k2[m] - k1[n]) * (h - d)) +
                1.0 / (k2[m] + k1[n]) * np.sin((k2[m] + k1[n]) * (h - d))
            )

    return C


In [18]:
def COEF3(h, d, nn, N_MAX, k2, n2):

    def EPSL(m):
      if m == 0:
        return 1.0
      elif m > 0:
        return 2.0
      else:
        print('error at epsl: m=', m)
        return 0.0

    C = np.empty((N_MAX + 1, nn + 1), dtype=float)

    k3 = np.empty(nn + 1, dtype=float)
    n3 = np.empty(nn + 1, dtype=float)

    for n in range(nn + 1):
        k3[n] = (2 * n + 1) * pi / (2 * h)
        n3[n] = 1.0 / np.sqrt(2.0)

    for m in range(nn + 1):
        for n in range(nn + 1):
            if abs(k2[m] - k3[n]) < EPSL(m):
                C[m, n] = 1.0 / (2 * h * n2[m] * n3[n]) * (
                    (h - d) + 1.0 / (k2[m] + k3[n]) * np.sin((k2[m] + k3[n]) * (h - d))
                )
            else:
                C[m, n] = 1.0 / (2 * h * n2[m] * n3[n]) * (
                    1.0 / (k2[m] - k3[n]) * np.sin((k2[m] - k3[n]) * (h - d)) +
                    1.0 / (k2[m] + k3[n]) * np.sin((k2[m] + k3[n]) * (h - d))
                )

    return C


In [19]:
def EPSL(m):
    if m == 0:
        return 1.0
    elif m > 0:
        return 2.0
    else:
        print('error at epsl: m=', m)
        return 0.0

def EVANESDF(X, D, N):
    pi = np.pi
    y = pi * N - D
    f0 = math.atan(X / y) - D
    f1 = X / (X * X + y * y) - 1.0
    f2 = 2.0 * X * y / (X * X + y * y) ** 2
    return D - f0 / f1 * (1.0 + 0.5 * f0 * f2 / (f1 * f1))

def EVANESD(X, N):
    pi = np.pi
    if X < 0.0 or N < 1:
        raise ValueError('error at evanes')

    u = 3.0 * X / (7.0 + 3.0 * X)
    d = 0.0159 + u * (0.1032 + u * (4.3152 - u * 2.8768))
    d = EVANESDF(X, d, 1)
    Wkn = [0.0] * N
    Wkn[0] = pi - d

    if N < 2:
        return Wkn

    for i in range(2, N + 1):
        d = d - pi * X / (X * X + pi * (i + 1) * (pi * i - d))
        d = EVANESDF(X, d, i)
        Wkn[i - 1] = pi * i - d

    return Wkn

def WNUMBER(X):
    if X < 0.0:
        raise ValueError('error at wnumber')

    b = [0.000000122, 0.073250017, -0.009899981, 0.002640863, -0.000829239, -0.000176411]
    c = [1.0, -0.33333372, -0.01109668, 0.01726435, 0.01325580, -0.00116594, 0.00829006, -0.01252603, 0.00404923]

    if X <= 2.0:
        a = 0.5 * X
        p = 1.0
        sum_val = c[0]

        for i in range(2, 10):
            p = p * a
            sum_val = sum_val + c[i-1] * p

        Wk0 = math.sqrt(X) / sum_val
    else:
        a = 0.5 * X * math.exp(4.0 - 2.0 * X)
        p = 1.0
        sum_val = b[0]

        for i in range(2, 7):
            p = p * a
            sum_val = sum_val + b[i-1] * p

        Wk0 = X + sum_val

    return Wk0

def COEF1(k, h, d, nn):

    k_1 = WNUMBER(k * h) / h
    #print(k_1)
    k1 = [ZI * k_1]
    #print(k1)
    #print(k * h)
    #print(math.sinh(2 * WNUMBER(k * h)))
    n1 = [math.sqrt((1.0 + math.sinh(2 * WNUMBER(k * h)) / (2 * WNUMBER(k * h))) / 2)]
    #print(k * h)
    #print(n1)

    wkn = EVANESD(k * h, nn)

    for n in range(1, nn + 1):
        k1.append(wkn[n - 1] / h)
        n1.append(math.sqrt((1.0 + math.sin(2 * wkn[n - 1]) / (2 * wkn[n - 1])) / 2))

    k2 = [n * PI / (h - d) for n in range(nn + 1)]
    n2 = [math.sqrt((1.0 - d / h) / EPSL(n)) for n in range(nn + 1)]

    return k1, k2, n1, n2, wkn


In [25]:
zgesv_n = int(N_MAX + 1)
zgesv_nrhs = 2
zgesv_lda = int(N_MAX + 1)
zgesv_ldb = int(N_MAX + 1)
zgesv_a = np.ndarray((zgesv_n, zgesv_n), dtype=np.complex128)
zgesv_b = np.ndarray((zgesv_n, zgesv_nrhs), dtype=np.complex128)
zgesv_ipiv = np.ndarray((zgesv_n), dtype=float)

def ZGESV(zgesv_n, zgesv_nrhs, zgesv_a, zgesv_lda, zgesv_ipiv, zgesv_b, zgesv_ldb):

    X = np.zeros((zgesv_n, zgesv_nrhs), dtype=np.complex128)
    INFO = 0
    try:
        for j in range(zgesv_nrhs):
            X[:, j], INFO = solve(zgesv_a, zgesv_b[:, j], overwrite_a=True, overwrite_b=True), 0
            if INFO != 0:
                break
    except Exception as e:
        INFO = -1  # An exception occurred
    return X, INFO

In [20]:
with open('barge.dat', 'w') as file_9:
    file_9.write(f"{rho:1.5e} ! fluid density, rho(Mg/m^3)\n")
    file_9.write(f"{g:1.5e} ! gravity acceleration, g(m/s^2)\n")
    file_9.write(f"{bb:1.5e} ! full width of the barge, B(m)\n")
    file_9.write(f"{d:1.5e} ! draft of the barge, d(m)\n")
    file_9.write(f"{h:1.5e} ! water depth, h(m)\n")
    file_9.write(f"{kg:1.5e} ! center of gravity, KG(m)\n")
    file_9.write(f"{rx:1.5e} ! radius of gyration, rx(m)\n")
    file_9.write(f"{df:1.5e} ! interval of frequencies, df(Hz)\n")
    file_9.write(f"{nf:12d} ! number of frequencies, nf\n")

# Open output files
file_10 = open('amass.csv', 'w')
file_11 = open('rdamp.csv', 'w')
file_12 = open('rdamp_far.csv', 'w')
file_13 = open('force.csv', 'w')
file_14 = open('force_far.csv', 'w')
file_15 = open('rao.csv', 'w')
file_16 = open('rt.csv', 'w')
file_17 = open('drift.csv', 'w')
file_18 = open('wnumber.csv', 'w')
file_19 = open('kochin.csv', 'w')
file_20 = open('check.csv', 'w')
file_21 = open('amass_inf.csv', 'w')

# Perform operations on the opened files, if needed

# Close the opened files when done
file_10.close()
file_11.close()
file_12.close()
file_13.close()
file_14.close()
file_15.close()
file_16.close()
file_17.close()
file_18.close()
file_19.close()
file_20.close()
file_21.close()

In [ ]:
for i in range(1, nf+1):
    f = df * i

    omega = 2 * PI * f
    #omega = round(2 * PI *f)

    k = omega**2 / g
    #k = round(omega**2 / g, 3)
    #print(k)

    #print(f)
    #print(omega)
    #print(k)

    COEF1(k, h, d, nn)
    COEF2(h, d, nn, N_MAX, k1, k2, n1, n2)
    CALFSA(h, b, nn, N_MAX, k2, c)
    MATRX(h, b, nn, N_MAX, k1, fs, fa, mats, mata, ass, aaa)
    CALGM(h, d, b, zg, nn, k1, k2, n1, n2)
    VECTR(h, b, nn, N_MAX, k2, C, g1, g2, g3, g4, g5, g6, a2, a3, a4)
    for j in range(nn + 1):
      zbs[j, 0] = ass[j]
      zbs[j, 1] = a3[j]
    ZGESV(zgesv_n, zgesv_nrhs, mats, N_MAX+1, zgesv_ipiv, zbs, N_MAX+1)

<ipython-input-139-2f08fef3abbe>:8: RuntimeWarning: divide by zero encountered in double_scalars
  C[m, n] = 1.0 / (2 * h * n2[m] * n1[n]) * (
<ipython-input-139-2f08fef3abbe>:9: RuntimeWarning: divide by zero encountered in cdouble_scalars
  1.0 / (k2[m] - k1[n]) * np.sin((k2[m] - k1[n]) * (h - d)) +
<ipython-input-139-2f08fef3abbe>:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  1.0 / (k2[m] - k1[n]) * np.sin((k2[m] - k1[n]) * (h - d)) +
<ipython-input-139-2f08fef3abbe>:10: RuntimeWarning: divide by zero encountered in cdouble_scalars
  1.0 / (k2[m] + k1[n]) * np.sin((k2[m] + k1[n]) * (h - d))
<ipython-input-139-2f08fef3abbe>:10: RuntimeWarning: invalid value encountered in cdouble_scalars
  1.0 / (k2[m] + k1[n]) * np.sin((k2[m] + k1[n]) * (h - d))
<ipython-input-142-134f2878b446>:12: RuntimeWarning: invalid value encountered in double_scalars
  zsum4 += k2[l] * h / np.tanh(k2[l] * b) * c[l, m] * c[l, n]
<ipython-input-151-6cfb18664060>:11: RuntimeWarning: divide by

In [ ]:
with open('barge.dat', 'w') as file_9:
    file_9.write(f"{rho:1.5e} ! fluid density, rho(Mg/m^3)\n")
    file_9.write(f"{g:1.5e} ! gravity acceleration, g(m/s^2)\n")
    file_9.write(f"{bb:1.5e} ! full width of the barge, B(m)\n")
    file_9.write(f"{d:1.5e} ! draft of the barge, d(m)\n")
    file_9.write(f"{h:1.5e} ! water depth, h(m)\n")
    file_9.write(f"{kg:1.5e} ! center of gravity, KG(m)\n")
    file_9.write(f"{rx:1.5e} ! radius of gyration, rx(m)\n")
    file_9.write(f"{df:1.5e} ! interval of frequencies, df(Hz)\n")
    file_9.write(f"{nf:12d} ! number of frequencies, nf\n")

# Define file names
file_names = [
    'barge.dat',
    'amass.csv',
    'rdamp.csv',
    'rdamp_far.csv',
    'force.csv',
    'force_far.csv',
    'rao.csv',
    'rt.csv',
    'drift.csv',
    'wnumber.csv',
    'kochin.csv',
    'check.csv',
    'amass_inf.csv'
]

# Open the files
file_handles = []
for file_name in file_names:
    file_handle = open(file_name, 'w')
    file_handles.append(file_handle)

# Write data to barge.dat
with open('barge.dat', 'w') as file:
    file.write(f'{rho:.5e} ! fluid density, rho(Mg/m^3)\n')
    file.write(f'{g:.5e} ! gravity acceleration, g(m/s^2)\n')
    file.write(f'{bb:.5e} ! full width of the barge, B(m)\n')
    file.write(f'{d:.5e} ! draft of the barge, d(m)\n')
    file.write(f'{h:.5e} ! water depth, h(m)\n')
    file.write(f'{kg:.5e} ! center of gravity, KG(m)\n')
    file.write(f'{rx:.5e} ! radius of gyration, rx(m)\n')
    file.write(f'{df:.5e} ! interval of frequencies, df(Hz)\n')
    file.write(f'{nf} ! number of frequencies, nf\n')

made by own